### Step-1: Preliminary setup

In [1]:
#Import gensim library
import gensim
from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser
from nltk.corpus import PlaintextCorpusReader
import nltk
import pandas as pd
import numpy as np
import time
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter


### Step-2: Load Dataset and Train Word2Vec model

In [2]:
start=time.time()
df = pd.read_csv("./amazon_food_review/Reviews.csv")
df.columns
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

Time taken 0.091440 min


### Extract Top Products with the most Reviews

In [3]:
 
top_product = Counter(df.ProductId).most_common()[0:1]

for i in range(len(top_product)):
    if i ==0:
        df2= df[df.ProductId == top_product[i][0]]
    else:
        df2 = pd.concat((df2,df[df.ProductId == top_product[i][0]]))
        
df2.shape

(913, 10)

### Check the count of Reviews in the dataset

In [4]:
from collections import Counter
print(Counter(df2.ProductId).most_common())
print(Counter(df2.Score))

[('B007JFMH8M', 913)]
Counter({5: 624, 4: 217, 3: 56, 2: 12, 1: 4})


### Tokenize the Reviews into Sentences - sent_tokenize

In [5]:
start=time.time()
#sents=list(df2.Text)
sents = df2.Text
sent_lst = []
for line in sents:
    #print(line)
    for token_line in sent_tokenize(line):
        #print(token_line)
        sent_lst.append([token_line])
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

len(sent_lst)

Time taken 0.005020 min


3667

### Sentences - Word_tokenize

In [6]:
from random import shuffle

start=time.time()
sent_lst2 = []
#unigram_lst=[]
for line in sent_lst:
    #print(word_tokenize(line[0]))
    sent_lst2.append(word_tokenize(line[0]))
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

shuffle(sent_lst2)
sent_train = sent_lst2[0: round(0.64*(len(sent_lst2)))] #SET 64% OF TRAINING DATA
len(sent_train)

Time taken 0.014086 min


2347

### Step-4: hand-write some patterns

In [7]:
#regex grammar
grammar = r"""
JN:{<RB.*|JJ.*>+<.>*}



NP:{<EX|DT|PRP\$>?<RB>*<JN|VB.*>+<NN.*>+<.>*}

cla1a:{<DT|PRP>?<NN.*>+<VB.*>+<JN>+<JJ.*>*<.>*}
        
cla1b:{<cla1a>+<,|CC>+<JN|NN.*>+<JN|NN.*>*<.>*}
        {<cla1a>+<,|CC>+<cla1a>+<.>*}

cla2a:{<DT>?<NN.*|PRP.*>+<IN|VB.*>+<DT>*<NP|NN.*>+<NP|NN.*|JN>*<.>*}

cla2b:{<DT>?<cla2a>+<,|CC>+<DT>*<JN|NN.*|NP>+<.>*}
        {<DT>?<cla2a>+<,|CC>+<cla2a>+<.>*}
        
    
        
cla3: {<DT>?<NN.*>+<NP>+<JN>*<.>*<IN>*<NP>*}

cla4: {<cla1a>+<VB.*>+<JN>*<.>*}

cla4b: {<cla4>+<PRP.*>+<NN.*>+<TO>*<VB.*>*<.>*}


cla5: {<NN.*|PRP.*><NP>+<IN>+<NP>+<.>*}
            {<NN.*|PRP.*><NP>+<IN>+<NP>*<.>*}

cla6: {<PRP>*<MD>*<JN>*<VB.*>+<DT>+<TO>+<NP>+<.>*}

cla7: {<WP>+<PRP>*<MD>+<JN>*<VB.*>+<PRP>+<JN>+<.>*}

cla8: {<PRP>?<MD>*<JN>*<VB.*>+<DT>+<TO>+<PRP\$>+<NN.*>+<.>*}

cla9: {<DT>*<NN>+<MD>*<VB.*>+<DT>+<NN>+<JN>+<IN>+<CD>*<NN.*>*<.>*}

cla10: {<PRP>+<VB.*>+<JN>+<NP>+<IN>+<NP+>*<.>*}

cla11: {<CC>+<DT>*<NN.*>?<JN>+<VB.*>+<IN>+<PRP.*>*<NN>*<.>*}

cla12: {<CC>?<VB.*>+<IN>+<PRP.*>+<.>*}

cla13: {<CC>?<PRP.*>+<VB.*>+<JN>+<IN>+<DT|VB.*>*<JJ.*|JN|NN.*>*<.>*}

cla14:{<CC>?<PRP.*>*<MD>*<JN>*<VB.*><TO|IN>+<VB.*|JN>+<.>*}

cla15: {<CC>?<NN.*|PRP>+<VB.*>+<VB.*|JN>+<PRP.*|DT|WDT|NN>*<VB.*>*<.>*}

cla15b: {<cla15>+<CC>+<JN|JJ>+<.>*}

cla15c: {<cla15>+<TO>+<VB.*>+}

cla16:{<CC>?<DT>+<VB.*>*<JN|JJ.*|NP>+<JN|VB.*>*<.>*}

cla16b:{<cla16><,|CC>+<NN.*|NP>*<.>*}

cla16c:{<cla16b>+<PRP.*>+<JN>*<VB.*>+<.>*}

cla17: {<NP>+<WDT|WP.|WRB>*<PRP.*>*<VB.*>+<NP|JN>*<.>*}

cla17b: {<cla17>+<TO|VBG>+<VB.*>*<JN>*<.>*}

cla17c:{<cla17>+<PRP.*>+<JN | JJ>*}

cla18:{<DT>?<CD>+<IN>*<JN|NN.*>*}

cla18a:{<cla18>+<,|CC>+<cla18>+<.>*}

cla19: {<PRP.*>*<JN>*<WP>?<MD>*<VB.*>+<DT|PRP.*>*<NP|NN.*|PRP.*>+<.>*}

cla19b: {<cla18>+<cla19>+<.>*}

cla20: {<cla15>+<cla12>+<.>*}

cla21: {<PRP>*<MD>*<JN>+<TO>*<VB>+<DT>*<cla16|cla18>*}

cla22: {<VB.*>+<cla2a>+<WRB|WP|WDT>*<PRP>*<VB.*>*<.>*}

cla23:{<cla2a>+<WDT|WP>+<cla19>+}

cla24: {<NNP>+<JN>+<IN>+<DT>+<PRP>*<:|.>*}

cla24b:{<cla24>+<cla17>+}

cla25: {<NN.*>+<TO|PRP.*>*<NN.*>+<,|IN|CC>*<NN.*>*<IN|TO>*<VB.*>+<PRP.*|DT>*<:|.>*}

cla25b:{<cla25>+<,|CC>*<JN|NP>+<.|:>*}

cla26: {<PRP.*>+<NN.*>+<,|IN|CC>*<NN.*>*<VB.*>+<DT|PRP.*>*<JN>*<:|.>*}

cla27: {<cla15>+<IN>*<WRB|WDT|WP.*>*<JN>+<DT|NN.*>*<VB.*>+<:|.>*}

cla28:{<NP|NNP>+<,|CC>*<NP|JN>+}

cla29: {<WP>+<JN>*<PRP.*>+<MD>+<VB.*>*<:|.>*}
    
cla30: {<WP>+<JN>*<VB.*>+<cla21>+<:|.>*}

cla31: {<NN.*>+<,|CC|IN>*<JN|JJ|NN.*>*<,|CC|IN>*<NN.*>*<:|.>*}

cla32: {<NN.*|PRP.*>*<MD>+<VB.*>*<PRP.*|DT>+<JN>*<.|:>*}

cla33:{<cla19>+<DT|JN | RP>*<.|:>*}

cla34:{<PRP.*>+<cla18>+<cla3>*<cla19>+<JN>*<.|:>*}

cla35:{<DT>*<JJS>+<,|CC>*<JJS>*<NN.*><PRP.*|DT>+<JN>*<.|:>*}

cla36:{<cla2a>+<IN>*<DT>*<NN.*|NP|cla31>+<.|:>*}

cla37:{<DT|PRP.*>*<cla15>+<CC|IN|,>*<cla15>+<.|:>*}

cla38:{<DT>+<MD>+<VB.*>+<IN|DT>*<cla31>+<.|:>*}

cla39:{<NN.*|PRP.*>+<VB.*>*<PRP.*>+<JN>*<.|:>*}

cla40:{<NN.*|PRP.*>*<MD>*<VB.*>+<JN|cla16>+<.|:>*}

cla41:{<cla33>+<TO|IN>+<cla2a>+<.|:>*}

cla42:{<cla31>+<TO|IN>+<VB.*>+<TO|IN>+<NP>+<.|:>*}

cla43:{<DT>*<cla31>+<cla40>+<TO|IN>+<DT|PRP.*>*<cla31>*<.|:>*}

cla44:{<WP>+<cla40>+<IN>*<DT|PRP.*>+<NP|NN.*>+<.|:>*}
    
cla44b:{<WP>+<cla40>+<IN>+<cla18|DT|NN.*|cla31>+<.|:>*}

cla45:{<cla2b>+<cla40>+<.|:>*}

cla46:{<cla13>+<NP>+<.|:>*}

cla47:{<WRB>+<JN|VB.*>+<CC|IN>*<JN|VB.*>*<DT|PRP.*>*<VB.*>+<.|:>*}

cla48:{<cla17>+<IN>+<cla2a>+<.|:>*}

cla49:{<cla19>+<IN>+<PRP.*>*<cla18|cla31>+<.|:>*}

cla49b: {<cla49>+<cla3>+<.|:>*}

cla50: {<PRP.*>?<cla12>+<cla31>+<.|:>*}

cla51:{<CC>*<DT|PRP.*>*<cla31>+<cla19+>+<JN>*<.|:>*}

cla52:{<cla2a>+<MD>*<VB.*>+<WRB>*<JN>+<DT>*<PRP.*|cla31>*<VB.*>*<.|:>*}

cla53:{<cla31>+<cla39|VG.*>*<DT|cla31>*<cla33>+<.|:>*}

Quotation:{<:|.>*<\"|\'|\``|\''|\`>+<NN.*|VB.*|JN|NP|JJ|cla24|cla25|cla26|cla27|cla28|cla29|cla30|cla31>*<\"|\'|\``|\''|\`>+<:|.>*}
"""

In [8]:
############# Grammar: NP
#sent = "Nicely smell cookiess"
#sent = "tasty cookies"


############ cla1a
#sent = "The cookiess seemed pretty tasty"
#sent = "the cookiess were excellent"
#sent = "the cookies is very nice"
#sent = "the bottles are not too large"
#sent = "the bottles are too large"
#sent ="the cookies are not good"
#sent ="the cookies are not very good enough"
#sent ="the cookies are not very good"
#sent = "this cookie was surprisingly yummy."

############ cla1b
#sent = "cookies was tasty and soft"
#sent = " the cookies is pretty beautiful and the cookies is pretty soft"
#sent = " the cookies is big and the rasins is very soft"

############ cla2a
#sent="I love the food so much"
#sent = "I participated in a product review"


############ cla2b
#sent = "She likes this big cookie and soft cookie"
#sent = "She likes this cookies and the smell"


############cla3
#sent = "the cookies are way too small"
#sent = "The cookies are soft, chewy, yummy,"
#sent = "These cookies have perfect flavor, excellent moist/dry combination"

############cla4
#sent="the cookie did not smell good"


############ 4b
#sent="please don't waste your money "

############cla5
#sent = "I love cookies with many rasins"


########### 7
#sent= "who would buy it again??"


############ 13
#sent = "It goes down hill from there."
#sent = "They're are so soft, perfect for anytime"


############ 14
#sent = "I can't wait to try "
#sent="can't wait to buy more"

############ 15
#sent="It was great!!!"
#sent = "i love eating it"
#sent = "They're are so soft and perfect"

############ 15b
#sent="They are so soft and tasty!"

############ 15c
#sent = "it is great to buy it"

##########16
#sent = "This is my great best experience!"
#sent = "This is the high point."
#sent = "This was the most delightful cookie i ever tasted."

##########16b
#sent = "the best and freshest cookie"

##########16c
#sent = "the best and freshest cookie I ever had"


##########17
#sent = "this is bad food which hurt your healthy body"
#sent = "The high quality ingredients are a crowd pleaser."


##########17b


##########17c
#sent = "Oatmeal cookies are my favorite"

########## 18
#sent="6g of fat , 27g total carbs"

########## 19
#sent="what's the point?"
#sent="you'll love this snack ....."
#sent = "I highly recommend this cookie"
#sent = "loved the food"
#sent = "loved them"
#sent ="I love my cookies,"

######### 21
#sent = "too old to eat"
#sent="I would never try one again."
#sent="I would never try this one again."

######## 22
#sent="throw them in the garbage which they belong"
#sent="throw them in the garbage where they belong"

######## 23
#sent="I participated in a product review that included a sample"

########25


########25b


########26
#sent="My kids and i  loved these."


########27
#sent="I was surprised how soft the cookie was."
#sent="I was immediately surprised by how soft the cookie was "

########28
#sent = "Delicious and nutritious ....."
#sent = "good price and delicious"

########29
#sent="what else you can say!!!"

######## 30
#sent="what else is there to say...."

####### 31
#sent = "Soft, chewy and tasty."
#sent = "Highly recommend!"
#sent = "yummy and delicious."

####### 32
#sent="I would recommend these"

#######33
#sent = "we ate them all...!!!"
#sent = "we ate them up...!!!"
#sent= "should i buy it again??"

###### 34
#sent="My three year old kid loved it so much !!!"


######36
#sent = "I am a huge fan of the Quaker Chewy cookies!"
#sent = "I am a huge fan of the cookies!"
#sent = "I am a huge fan of the Chewy cookies!"
#sent = "I am a huge fan of Chewy cookies!"
#sent = "I am a huge fan of cookies!"

###### 37
#sent="The moist cookies are satisfying and they are not too sweet."

###### 40
#sent ="I will buy some more"

###### 41
#sent = "I recommend these to anyone with a sweet tooth"


###### 42
#sent = "Great to take as a quick breakfast "

###### 43
#sent = "This cookie would be perfect for breakfast"
#sent = "This cookie would be perfect for my family..."
#sent = "This cookie would be perfect for me"

###### 44
#sent = "What could be better than that?!"

###### 44b
#sent = "What could be better than this one ?!"
#sent = "What could be better than this product ?!"
#sent = "What could be better than these products ?!"
#sent = "What could be better than that?!"

###### 46
#sent = "they are more healthy than most other products."
#sent="they are more healther than other food"

###### 47
#sent = "how soft and fresh it tasted"
#sent="How good it tasted?"

###### 49
#sent="I even shared it with my brother"
#sent="I shared it with my child"

########50
#sent = "They taste like my grandmas homeade cookies.."

########51
#sent = "the kids love them!"
#sent="My  kid loved it so much "

########52
#sent = "I guess the younger folks will like how sweet the cookie is."
#sent=sent_lst[149][0]

########53
#sent="Thank you my mom buying me this!!!"
#sent="thanks for buying this"
#sent="Thanks for buying me this"


#print(sent)
#print()
#print(nltk.pos_tag(nltk.word_tokenize(sent)))
#print()
#sent =  nltk.word_tokenize(sent)
#print(sent)
review_lst=[]


for sent in sent_train:
    cp = nltk.RegexpParser(grammar)
    tree = cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":        print("NP", subtree.leaves()),review_lst.append(subtree.leaves())
        #if subtree.label() =="JN":         print("JN", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="Quotation":   print("Quotation", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla1a":         print("cla1a", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla1b":         print("cla1b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla2a":         print("cla2a", subtree.leaves()), review_lst.append(subtree.leaves())
        if subtree.label() =="cla2b":         print("cla2b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla3":         print("cla3", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla4":         print("cla4", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla4b":         print("cla4b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla5":         print("cla5", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla6":         print("cla6", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla7":         print("cla7", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla8":         print("cla8", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla9":         print("cla9", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla10":         print("cla10", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla11":         print("cla11", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla12":         print("cla12", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla13":         print("cla13", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla14":        print("cla14", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15":         print("cla15", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15b":         print("cla15b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15c":         print("cla15c", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16":         print("cla16", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16b":         print("cla16b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16c":         print("cla16c", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17":         print("cla17", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17b":         print("cla17b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17c":         print("cla17c", subtree.leaves()),review_lst.append(subtree.leaves())        
        if subtree.label() =="cla18":         print("cla18", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla18a":         print("cla18a", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla19":         print("cla19", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla19b":         print("cla19b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla20":         print("cla20", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla21":         print("cla21", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla22":         print("cla22", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla23":         print("cla23", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla24":         print("cla24", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla24b":         print("cla24b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla25":         print("cla25", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla25b":         print("cla25b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla26":         print("cla26", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla27":         print("cla27", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla28":         print("cla28", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla29":         print("cla29", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla30":         print("cla30", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla31":         print("cla31", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla32":         print("cla32", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla33":         print("cla33", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla34":         print("cla34", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla35":         print("cla35", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla36":         print("cla36", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla37":         print("cla37", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla38":         print("cla38", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla39":         print("cla39", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla40":       print("cla40",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla41":       print("cla41",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla42":       print("cla42",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla43":       print("cla43",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla44":       print("cla44",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla44b":       print("cla44b",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla45":       print("cla45",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla46":       print("cla46",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla47":       print("cla47",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla48":       print("cla48",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla49":       print("cla49",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla49b":       print("cla49b",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla50":       print("cla50",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla51":       print("cla51",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla52":       print("cla52",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla53":       print("cla53",subtree.leaves()),review_lst.append(subtree.leaves())
print("Number of Extracted Segments",len(review_lst))

cla17 [('These', 'DT'), ('soft', 'JJ'), ('oatmeal', 'NN'), ('raisin', 'NN'), ('cookies', 'NNS'), ('are', 'VBP'), ('actually', 'RB'), ('quite', 'RB'), ('good', 'JJ'), ('.', '.')]
NP [('These', 'DT'), ('soft', 'JJ'), ('oatmeal', 'NN'), ('raisin', 'NN'), ('cookies', 'NNS')]
cla33 [('I', 'PRP'), ('even', 'RB'), ('shared', 'VBD'), ('it', 'PRP')]
cla19 [('I', 'PRP'), ('even', 'RB'), ('shared', 'VBD'), ('it', 'PRP')]
cla18 [('11', 'CD'), ('month', 'NN'), ('old', 'JJ'), (',', ',')]
cla2a [('I', 'PRP'), ("'m", 'VBP'), ('a', 'DT'), ('pretty', 'RB'), ('big', 'JJ'), ('health', 'NN'), ('nut', 'NN')]
NP [('a', 'DT'), ('pretty', 'RB'), ('big', 'JJ'), ('health', 'NN'), ('nut', 'NN')]
cla33 [('I', 'PRP'), ('feed', 'VBP'), ('my', 'PRP$'), ('daughter', 'NN'), ('.', '.')]
cla19 [('I', 'PRP'), ('feed', 'VBP'), ('my', 'PRP$'), ('daughter', 'NN'), ('.', '.')]
cla15 [('you', 'PRP'), ('start', 'VBP'), ('eating', 'VBG'), ('this', 'DT'), (',', ',')]
cla33 [('you', 'PRP'), ("'ll", 'MD'), ('finish', 'VB'), ('it', 

cla31 [('box', 'NN'), ('once', 'IN')]
cla15 [('they', 'PRP'), ('are', 'VBP'), ('available', 'JJ'), ('.', '.')]
cla15c [('I', 'PRP'), ('was', 'VBD'), ('very', 'RB'), ('pleased', 'JJ'), ('to', 'TO'), ('give', 'VB')]
cla15 [('I', 'PRP'), ('was', 'VBD'), ('very', 'RB'), ('pleased', 'JJ')]
cla31 [('kids', 'NNS')]
cla36 [('a', 'DT'), ('quick', 'NN'), ('on', 'IN'), ('the', 'DT'), ('go', 'NN'), ('healthy', 'JJ'), ('snack', 'NN'), ('before', 'IN'), ('soccer', 'NN'), ('and', 'CC'), ('acne', 'JJ'), ('practice', 'NN'), ('.', '.')]
cla2a [('a', 'DT'), ('quick', 'NN'), ('on', 'IN'), ('the', 'DT'), ('go', 'NN'), ('healthy', 'JJ'), ('snack', 'NN')]
NP [('healthy', 'JJ'), ('snack', 'NN')]
cla31 [('soccer', 'NN'), ('and', 'CC')]
NP [('acne', 'JJ'), ('practice', 'NN'), ('.', '.')]
cla53 [('spices', 'NNS'), ('make', 'VBP'), ('me', 'PRP')]
cla31 [('spices', 'NNS')]
cla33 [('make', 'VBP'), ('me', 'PRP')]
cla19 [('make', 'VBP'), ('me', 'PRP')]
cla50 [('think', 'VB'), ('of', 'IN'), ('my', 'PRP$'), ('grandma',

cla16 [('This', 'DT'), ('was', 'VBD'), ('the', 'DT'), ('first', 'JJ'), ('product', 'NN')]
NP [('the', 'DT'), ('first', 'JJ'), ('product', 'NN')]
cla18 [('the', 'DT'), ('5', 'CD')]
cla14 [('sent', 'VBN'), ('to', 'TO'), ('test', 'VB')]
cla5 [('it', 'PRP'), ('lasted', 'VBD'), ('less', 'RBR'), ('then', 'RB'), ('ten', 'VB'), ('minutes', 'NNS'), ('in', 'IN')]
NP [('lasted', 'VBD'), ('less', 'RBR'), ('then', 'RB'), ('ten', 'VB'), ('minutes', 'NNS')]
cla31 [('home', 'NN'), ('.', '.')]
cla17 [('any', 'DT'), ('sweet', 'JJ'), ('tooth', 'NN'), ('craving', 'VBG')]
NP [('any', 'DT'), ('sweet', 'JJ'), ('tooth', 'NN')]
cla31 [('calories', 'NNS')]
NP [('other', 'JJ'), ('desserts', 'NNS'), ('.', '.')]
cla2a [('He', 'PRP'), ('had', 'VBD'), ('no', 'DT'), ('clue', 'NN'), ('there', 'EX'), ('were', 'VBD'), ('raisins', 'NNS')]
NP [('there', 'EX'), ('were', 'VBD'), ('raisins', 'NNS')]
cla13 [('They', 'PRP'), ('were', 'VBD'), ('just', 'RB'), ('that', 'IN'), ('yummy', 'NN'), ('(', '('), (':', ':')]
cla40 [('I', 

cla31 [('Sorry', 'NN'), ('!', '.')]
NP [('a', 'DT'), ('great', 'JJ'), ('special', 'JJ'), ('treat', 'NN')]
NP [('whole', 'JJ'), ('grains', 'NNS')]
cla33 [('get', 'VB'), ('that', 'DT'), ('extra', 'JJ'), ('fiber', 'NN'), ('we', 'PRP'), ('all', 'DT')]
cla19 [('get', 'VB'), ('that', 'DT'), ('extra', 'JJ'), ('fiber', 'NN'), ('we', 'PRP')]
NP [('that', 'DT'), ('extra', 'JJ'), ('fiber', 'NN')]
cla33 [('I', 'PRP'), ('buy', 'VBP'), ('them', 'PRP')]
cla19 [('I', 'PRP'), ('buy', 'VBP'), ('them', 'PRP')]
NP [('quick', 'JJ'), ('snacks', 'NNS'), (',', ',')]
cla43 [('taste', 'NN'), ('sweet', 'NN'), ('and', 'CC'), ('delicious', 'JJ'), ('but', 'CC'), ('are', 'VBP'), ('healthier', 'JJR'), ('than', 'IN')]
cla31 [('taste', 'NN'), ('sweet', 'NN'), ('and', 'CC'), ('delicious', 'JJ'), ('but', 'CC')]
cla40 [('are', 'VBP'), ('healthier', 'JJR')]
NP [('other', 'JJ'), ('options', 'NNS'), ('.', '.')]
cla31 [('i', 'NN')]
cla14 [('will', 'MD'), ('continue', 'VB'), ('to', 'TO'), ('buy', 'VB')]
cla28 [('the', 'DT'), (

cla19 [('would', 'MD'), ('love', 'VB'), ('it', 'PRP'), (',', ',')]
cla40 [('was', 'VBD'), ('uber', 'RB'), ('pleased', 'JJ')]
cla12 [('see', 'VB'), ('that', 'IN'), ('my', 'PRP$')]
cla19b [('2', 'CD'), ('year', 'NN'), ('old', 'JJ'), ('loves', 'VBZ'), ('it', 'PRP')]
cla18 [('2', 'CD'), ('year', 'NN'), ('old', 'JJ')]
cla19 [('loves', 'VBZ'), ('it', 'PRP')]
cla3 [('The', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('soft', 'JJ'), (',', ','), ('chewy', 'NN'), (',', ',')]
NP [('are', 'VBP'), ('soft', 'JJ'), (',', ','), ('chewy', 'NN'), (',', ',')]
cla31 [('filling', 'NN'), ('.', '.')]
NP [('were', 'VBD'), ('moist', 'NN')]
cla31 [('yummy', 'NN'), ('.', '.')]
cla15 [('and', 'CC'), ('they', 'PRP'), ('are', 'VBP'), ('not', 'RB')]
cla31 [('snack', 'NN'), ('health', 'NN'), ('wise', 'NN'), ('!', '.')]
cla31 [('products', 'NNS'), ('.', '.')]
cla53 [('Yumm', 'NNP'), ('yumm', 'PRP'), ('yumm', 'VBP'), ('we', 'PRP')]
cla31 [('Yumm', 'NNP')]
cla33 [('yumm', 'PRP'), ('yumm', 'VBP'), ('we', 'PRP')]
cla19 [('

...

cla33 [('could', 'MD'), ('be', 'VB'), ('a', 'DT'), ('few', 'JJ'), ('more', 'JJR'), ('raisins', 'NNS')]
cla19 [('could', 'MD'), ('be', 'VB'), ('a', 'DT'), ('few', 'JJ'), ('more', 'JJR'), ('raisins', 'NNS')]
NP [('a', 'DT'), ('few', 'JJ'), ('more', 'JJR'), ('raisins', 'NNS')]
NP [('it.', 'JJ'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Thank', 'NNP')]
cla14 [('you', 'PRP'), ('influenster', 'VBP'), ('for', 'IN'), ('allowing', 'VBG')]
cla33 [('try', 'VB'), ('this', 'DT'), ('product', 'NN'), ('.', '.')]
cla19 [('try', 'VB'), ('this', 'DT'), ('product', 'NN'), ('.', '.')]
cla15 [('They', 'PRP'), ('are', 'VBP'), ('so', 'RB'), ('soft', 'JJ')]
cla31 [('chewy', 'NN'), ('.', '.')]
cla2a [('I', 'PRP'), ('recieved', 'VBD'), ('this', 'DT'), ('cookie', 'NN')]
cla36 [('the', 'DT'), ('mom', 'NN'), ('vox', 'NN'), ('box', 'NN'), ('on', 'IN'), ('[', 'NN'), ('...', ':'), (']', 'NN'), ('and', 'CC'), ('after', 'IN')]
cla2a [('the', 'DT')

cla15 [('product', 'NN'), ('was', 'VBD'), ('provided', 'VBN')]
cla31 [('Influenster', 'NNP'), ('.', '.')]
cla31 [('YUM', 'NN'), ('.', '.')]
cla2a [('I', 'PRP'), ('took', 'VBD'), ('my', 'PRP$'), ('first', 'JJ'), ('bite', 'NN')]
NP [('my', 'PRP$'), ('first', 'JJ'), ('bite', 'NN')]
cla2a [('I', 'PRP'), ('was', 'VBD'), ('in', 'IN'), ('love', 'NN'), (',', ',')]
cla1b [('the', 'DT'), ('cookie', 'NN'), ('was', 'VBD'), ('soft', 'JJ'), ('and', 'CC'), ('moist', 'NN')]
cla1a [('the', 'DT'), ('cookie', 'NN'), ('was', 'VBD'), ('soft', 'JJ')]
cla37 [('it', 'PRP'), ('was', 'VBD'), ('gone', 'VBN'), ('I', 'PRP'), ('found', 'VBD'), ('myself', 'PRP'), ('wanting', 'VBG'), ('more', 'RBR'), ('.', '.')]
cla15 [('it', 'PRP'), ('was', 'VBD'), ('gone', 'VBN'), ('I', 'PRP'), ('found', 'VBD')]
cla15 [('myself', 'PRP'), ('wanting', 'VBG'), ('more', 'RBR'), ('.', '.')]
cla17 [('These', 'DT'), ('new', 'JJ'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('cookies', 'NNS'), ('are', 'VBP'), ('amazing', 'VBG'),

cla31 [('TOTALLY', 'NNP'), ('DIFFERENT', 'NNP'), ('!', '.')]
cla31 [('<', 'NN'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('chewy', 'NN'), (',', ',')]
NP [('soft-baked', 'JJ'), ('cookie', 'NN'), (',', ',')]
cla31 [('Healthier', 'NNP'), ('alternative', 'NN'), (',', ',')]
cla3 [('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Individually', 'NNP'), ('packaged', 'VBD'), ('/', 'RB'), ('Good', 'NNP'), ('for', 'IN'), ('on', 'IN')]
NP [('packaged', 'VBD'), ('/', 'RB'), ('Good', 'NNP')]
cla31 [('go', 'NN'), ('!', '.')]
cla36 [('<', 'NN'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'VBZ'), ('The', 'DT'), ('Verdict', 'NN'), (':', ':'), ('<', 'JJ'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Great', 'NNP'), ('Healthier', 'NNP'), ('snack', 'NN')]
cla2a [('<', 'NN'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'VBZ'), ('The', 'DT'), ('Verdict', 'NN'), (':', ':')]
NP [('<', 'JJ'), ('br', 'NN'), ('

cla53 [('kids', 'NNS'), ('and', 'CC'), ('i', 'VB'), ('enjoyed', 'VBP'), ('the', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('cookies', 'NNS'), ('.', '.')]
cla31 [('kids', 'NNS'), ('and', 'CC')]
cla33 [('i', 'VB'), ('enjoyed', 'VBP'), ('the', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('cookies', 'NNS'), ('.', '.')]
cla19 [('i', 'VB'), ('enjoyed', 'VBP'), ('the', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('cookies', 'NNS'), ('.', '.')]
NP [('couldnt', 'VBP'), ('wait', 'NN')]
cla40 [('buy', 'VB'), ('more', 'JJR'), ('!', '.')]
cla2a [('I', 'PRP'), ('got', 'VBD'), ('the', 'DT'), ('influenster', 'NN'), ('box', 'NN')]
cla5 [('it', 'PRP'), ('had', 'VBD'), ('several', 'JJ'), ('products', 'NNS'), ('in', 'IN')]
NP [('had', 'VBD'), ('several', 'JJ'), ('products', 'NNS')]
cla31 [('son', 'NN'), ('and', 'CC')]
cla33 [('loved', 'VBD'), ('them', 'PRP'), ('.', '.')]
cla19 [('loved', 'VBD'), ('them', 'PRP'), ('.', '.')]
cla2a [('I', 'PRP'), ('receiv

cla31 [('daughter', 'NN')]
cla40 [('is', 'VBZ'), ('almost', 'RB')]
cla18 [('2', 'CD')]
cla4 [('the', 'DT'), ('cookie..', 'NN'), ('i', 'VBZ'), ('only', 'RB'), ('got', 'VBD')]
cla1a [('the', 'DT'), ('cookie..', 'NN'), ('i', 'VBZ'), ('only', 'RB')]
cla33 [('get', 'VB'), ('a', 'DT'), ('little', 'JJ'), ('piece', 'NN')]
cla19 [('get', 'VB'), ('a', 'DT'), ('little', 'JJ'), ('piece', 'NN')]
NP [('a', 'DT'), ('little', 'JJ'), ('piece', 'NN')]
cla5 [('she', 'PRP'), ('didnt', 'VBD'), ('want', 'NN'), ('at', 'IN')]
NP [('didnt', 'VBD'), ('want', 'NN')]
cla31 [('end', 'NN'), ('but', 'CC')]
cla41 [('i', 'JJ'), ('put', 'VBD'), ('it', 'PRP'), ('on', 'IN'), ('my', 'PRP$'), ('grocery', 'NN'), ('list', 'NN'), ('for', 'IN'), ('this', 'DT'), ('week', 'NN'), ('i', 'NN'), ('cant', 'VBP'), ('wait', 'NN')]
cla33 [('i', 'JJ'), ('put', 'VBD'), ('it', 'PRP')]
cla19 [('i', 'JJ'), ('put', 'VBD'), ('it', 'PRP')]
cla2a [('my', 'PRP$'), ('grocery', 'NN'), ('list', 'NN'), ('for', 'IN'), ('this', 'DT'), ('week', 'NN'), (

In [15]:
test_lst=[]
sent="it is A++++"
test_lst.append(nltk.pos_tag(word_tokenize(sent)))
test_lst

[[('it', 'PRP'), ('is', 'VBZ'), ('A++++', 'NNP')]]

### To CSV

In [58]:
Ext_sen_df = pd.DataFrame(review_lst)
Ext_sen_df.to_csv("Ext_sen.csv", index=False, header=False)


### Filtering out all the segments without Adverb or Adjectives or Verbs

In [9]:
review_lst2=[]
for tups in review_lst:
#for tups in test_lst:
    #print(tups)
    filter_cnt=0
    for tup in tups:
        if tup[1] in ["JJ","JJR","JJS","RB","RBR","RBS","VB","VBD","VBG","VBN","VBP", "VBZ"]:
            #print(tup[1])
            filter_cnt+=1
    
    if filter_cnt>0:
        review_lst2.append(tups)
print("Number of Extracted Segments after filtering",len(review_lst2))

Number of Extracted Segments after filtering 7250


In [140]:
review_lst2

[[('these', 'DT'),
  ('soft', 'JJ'),
  ('oatmeal', 'NN'),
  ('raisin', 'NN'),
  ('cookies', 'NNS'),
  ('are', 'VBP'),
  ('actually', 'RB'),
  ('quite', 'RB'),
  ('good', 'JJ'),
  ('.', '.')],
 [('these', 'DT'),
  ('soft', 'JJ'),
  ('oatmeal', 'NN'),
  ('raisin', 'NN'),
  ('cookies', 'NNS')],
 [('i', 'PRP'), ('even', 'RB'), ('shared', 'VBD'), ('it', 'PRP')],
 [('i', 'PRP'), ('even', 'RB'), ('shared', 'VBD'), ('it', 'PRP')],
 [('11', 'CD'), ('month', 'NN'), ('old', 'JJ'), (',', ',')],
 [('i', 'PRP'),
  ("'m", 'VBP'),
  ('a', 'DT'),
  ('pretty', 'RB'),
  ('big', 'JJ'),
  ('health', 'NN'),
  ('nut', 'NN')],
 [('a', 'DT'),
  ('pretty', 'RB'),
  ('big', 'JJ'),
  ('health', 'NN'),
  ('nut', 'NN')],
 [('i', 'PRP'),
  ('feed', 'VBP'),
  ('my', 'PRP$'),
  ('daughter', 'NN'),
  ('.', '.')],
 [('i', 'PRP'),
  ('feed', 'VBP'),
  ('my', 'PRP$'),
  ('daughter', 'NN'),
  ('.', '.')],
 [('you', 'PRP'),
  ('start', 'VBP'),
  ('eating', 'VBG'),
  ('this', 'DT'),
  (',', ',')],
 [('you', 'PRP'), ("'ll", '

### Functions to get the Wordnet Tag  and lemmatized words forms

In [10]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

def to_wntag(pos):
    """
    Given a Penn tag, returns the corresponding Wordnet pos tag.
    Note that WordNet only contain nouns/verbs/adjectives/adverbs.
    """
    ch = pos[0]
    if ch == 'N':
        return wn.NOUN #noun
    elif ch == 'V':
        return wn.VERB # verb
    elif ch == 'J':
        return wn.ADJ # adjective
    elif ch == 'R':
        return wn.ADV # adverb
    else:
        return None

def wn_lemmatize(token, wntag):
    """
    Applies WordNet morpher (which requires a pos) and 
    returns the morphological lemma form, or None.
    """
    return wn.morphy(token, wntag)

### Step-5: Parse segments to get the Synset[0] and Synset[ALL]

In [11]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
#df_lst = [0 for item in range(len(review_lst2))]
df_syn0_lst = []
df_syn_all_lst = []
df_tup_lst = []

cnt=0
#for tups in review_lst[0:10]:
for tups in review_lst2:
    if len(tups)<3:
        continue
    #print(tups)
    lst=[]
    tot_pscore_syn0=0
    tot_pscore_syn_all = 0
    word_cnt=0
    
    for i in range(len(tups)):
        #print("tups[i]",tups[i])
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        #print("w",w)
        #print("tag",tag)
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print("1")
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                #print("2")
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                #print("3")
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            #print(synsets)
            #print(synset)
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            #print(swn_synset)
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            #print(pscore)
            tot_pscore_syn0+=pscore
            
            tot_pscore_syn=0
            if len(synsets)>1:
                for synset in synsets:
                    swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                    #print(swn_synset)
                    pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                    #print("pscore",pscore)
                    tot_pscore_syn += pscore
                tot_pscore_syn_all += (tot_pscore_syn/len(synsets))
                #print("tot_pscore_syn_all",tot_pscore_syn_all)
            else: tot_pscore_syn_all += tot_pscore_syn0

            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
    
    if word_cnt == 0:
        df_syn0_lst.append(0)
        df_syn_all_lst.append(0)
        #print("Word count",word_cnt)
        df_tup_lst.append(tups)
        cnt+=1
    else:
        df_syn0_lst.append(tot_pscore_syn0/word_cnt)
        df_syn_all_lst.append(tot_pscore_syn_all/word_cnt)
        #print("Word count",word_cnt)
        df_tup_lst.append(tups)
        cnt+=1
        
print(len(df_syn0_lst), len(df_syn_all_lst), len(df_tup_lst))
    


6593 6593 6593


In [271]:
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
pd.DataFrame(list(zip(df_tup_lst,df_syn0_lst,df_syn_all_lst )), columns=["segments","syn0", "syn_all"])
#df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
#df2= pd.DataFrame(df_lst2)
#df = pd.concat((df1,df2), axis=1)
#df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 

#                   ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

,syn0,syn_all,segments
0,0.00000,0.145833,"[(very, RB), (great, JJ), (product, NN)]"
1,-0.25000,-0.083333,"[(daughter, NN), (stole, VBD), (it, PRP)]"
2,0.00000,0.041667,"[(ate, VB), (it, PRP), (,, ,)]"
3,0.00000,0.041667,"[(ate, VB), (it, PRP), (,, ,)]"
4,0.21875,0.140253,"[(guess, VBP), (it, PRP), ('s, VBZ), (a, DT), ..."


In [203]:
test_lst= []

sent = "NO comemnt"
sent = word_tokenize(sent)
#print(sent)
tree= cp.parse(nltk.pos_tag(sent))
cnt=0
for subtree in tree.subtrees():
    print(cnt)
    #print(subtree.leaves())
    test_lst.append(subtree.leaves())
    cnt+=1
test_lst[0]

0
1


[('NO', 'DT'), ('comemnt', 'NN')]

In [259]:
Counter(df_tup_lst[0]).most_common()[0][]

1

### Extracting more features from the Segments (e.g. Word structures, Part of Speech)

In [14]:
repeat_word_cnt_lst=[]#1
no_cnt_lst=[]#2
never_cnt_lst=[]#3
quite_cnt_lst=[]#4
but_cnt_lst=[]#5
very_cnt_lst=[]#6
more_cnt_lst=[]#7
br_cnt_lst=[]#8
else_cnt_lst=[]#9
must_cnt_lst=[]#10
many_cnt_lst=[]#11
up_cnt_lst=[]#12
so_cnt_lst=[]#13
all_cnt_lst=[]#14
dots_cnt_lst=[]#15
exclamation_mark_cnt_lst=[]#16
question_mark_cnt_lst=[]#17
not_cnt_lst=[]#18
er_cnt_lst=[]#19
est_cnt_lst=[]#20
ous_cnt_lst=[]#21
ly_cnt_lst=[]#22
dy_cnt_lst=[]#23
ful_cnt_lst=[]#24
ed_cnt_lst=[]#25
less_cnt_lst=[]#26
re_cnt_lst=[]#27
plus_cnt_lst=[]#28
minus_cnt_lst=[]#29
lol_cnt_lst=[]#30


CC_cnt_lst=[]#101
CD_cnt_lst=[]#102
DT_cnt_lst=[]#103
IN_cnt_lst=[]#104
JJ_cnt_lst=[]#105
JJR_cnt_lst=[]#106
JJS_cnt_lst=[]#107
MD_cnt_lst=[]#108
NN_cnt_lst=[]#109
NNS_cnt_lst=[]#110
NNP_cnt_lst=[]#111
NNPS_cnt_lst=[]#112
POS_cnt_lst=[]#113
PDT_cnt_lst=[]#114
PRP_cnt_lst=[]#115
PRP_dol_cnt_lst=[]#116
RB_cnt_lst=[]#117
RBR_cnt_lst=[]#118
RBS_cnt_lst=[]#119
TO_cnt_lst=[]#120
VB_cnt_lst=[]#121
VBD_cnt_lst=[]#122
VBG_cnt_lst=[]#123
VBN_cnt_lst=[]#124
VBP_cnt_lst=[]#125
VBZ_cnt_lst=[]#126
WDT_cnt_lst=[]#127
WP_cnt_lst=[]#128
WP_dol_cnt_lst=[]#129
WRB_cnt_lst=[]#130




pre_item=()
new_item=()

for tups in df_tup_lst:
#for tups in test_lst[0]:
    print(tups)
    
    #Counting Variables Initialization(Counting Specific Words in the segments) 
    repeat_word_cnt =0#1
    no_cnt =0#2
    never_cnt =0#3
    quite_cnt =0#4
    but_cnt =0#5
    very_cnt =0#6
    more_cnt =0#7
    br_cnt =0#8
    else_cnt =0#9
    must_cnt =0#10
    many_cnt =0#11
    up_cnt =0#12
    so_cnt =0#13
    all_cnt =0#14
    dots_cnt =0#15
    exclamation_mark_cnt =0#16
    question_mark_cnt =0#17
    not_cnt =0#18
    er_cnt =0#19
    est_cnt =0#20
    ous_cnt =0#21
    ly_cnt =0#22
    dy_cnt =0#23
    ful_cnt =0#24
    ed_cnt =0#25
    less_cnt =0#26
    re_cnt=0#27
    plus_cnt=0#28
    minus_cnt=0#29
    lol_cnt=0#30
    
    #Part-OF-Speech Tagging variable initialization
    CC_cnt =0#101
    CD_cnt =0#102
    DT_cnt =0#103
    IN_cnt =0#104
    JJ_cnt =0#105
    JJR_cnt =0#106
    JJS_cnt =0#107
    MD_cnt =0#108
    NN_cnt =0#109
    NNS_cnt =0#110
    NNP_cnt =0#111
    NNPS_cnt =0#112
    POS_cnt =0#113
    PDT_cnt =0#114
    PRP_cnt =0#115
    PRP_dol_cnt =0#116
    RB_cnt =0#117
    RBR_cnt =0#118
    RBS_cnt =0#119
    TO_cnt =0#120
    VB_cnt =0#121
    VBD_cnt =0#122
    VBG_cnt =0#123
    VBN_cnt =0#124
    VBP_cnt =0#125
    VBZ_cnt =0#126
    WDT_cnt =0#127
    WP_cnt =0#128
    WP_dol_cnt =0#129
    WRB_cnt =0#130
    
    
    for item in tups:
        pre_item=item
        
        #1
        #Check Consecutive repeated word 
        if new_item == pre_item:
            repeat_word_cnt+=1
        else:
            new_item = pre_item
        
        #Counting Specific Words in the segments
        
        #2
        if item[0].lower() == "no":
            print("no")
            no_cnt+=1
        #3    
        if item[0].lower() == "never":
            print("never")
            never_cnt+=1
        #4    
        if item[0].lower() == "quite":
            print("quite")
            quite_cnt+=1
        #5
        if item[0].lower() == "but":
            print("but")
            but_cnt+=1
    
        #6
        if item[0].lower() == "very":
            print("very")
            very_cnt+=1
        
        #7
        if item[0].lower() == "more":
            print("more")
            more_cnt+=1
                
        #8
        if item[0].lower() == "br":
            print("br")
            br_cnt+=1
        
        #9
        if item[0].lower() == "else":
            print("else")
            else_cnt+=1
        
        #10
        if item[0].lower() == "must":
            print("must")
            must_cnt+=1
        
        #11
        if item[0].lower() == "many":
            print("many")
            many_cnt+=1
                
        #12
        if item[0].lower() == "up":
            print("up")
            up_cnt+=1
        
        #13
        if item[0].lower() == "so":
            print("so")
            so_cnt+=1
        
        #14
        if item[0].lower() == "all":
            print("all")
            all_cnt+=1
        
        #15
        if item[0] == "...":
            print("...")
            dots_cnt+=1
        
        #16
        if item[0] == "!":
            print("!")
            exclamation_mark_cnt+=1
        
        #17
        if item[0] == "?":
            print("?")
            question_mark_cnt+=1
        
        #18
        if item[0] == "not":
            print("not")
            not_cnt+=1
            
            
        #Counting Specific pattern of last several characters from Adverbs and Adjuectives    
            
        #19
        if (item[0][-2:] == "er") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("er")
            er_cnt+=1
        #20     
        if (item[0][-3:] == "est") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("est")
            est_cnt+=1
        #21
        if (item[0][-4:] == "ous") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ous")
            ous_cnt+=1
        #22
        if (item[0][-2:] == "ly") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ly")
            ly_cnt+=1
        #23     
        if (item[0][-2:] == "dy") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("dy")
            dy_cnt+=1
        #24
        if (item[0][-2:] == "ful") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ful")
            ful_cnt+=1
        #25 
        if (item[0][-2:] == "ed") and ( item[1]=="VBD" or item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ed")
            ed_cnt+=1
        #26
        if (item[0][-4:] == "less") and ( item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("less")
            less_cnt+=1
        
        #27
        if (item[0][0:2] == "re"):
            print("re")
            re_cnt+=1
            
        #28
        for letter in item[0]:
            if letter is "+":
                plus_cnt+=1 
                
        #29
        for letter in item[0]:
            if letter is "-":
                minus_cnt+=1
        
        #30
        if item[0] is "lol":
            lol_cnt+=1#30
        
        print(item[1])
        #Counting  Part-Of-Speech
        #101
        if item[1] =="CC":
            print("CC")
            CC_cnt+=1
        #102
        if item[1] =="CD":
            print("CD")
            CD_cnt+=1
        #103
        if item[1] =="DT":
            print("DT")
            DT_cnt+=1
        #104
        if item[1] =="IN":
            print("IN")
            IN_cnt+=1
        #105
        if item[1] =="JJ":
            print("JJ")
            JJ_cnt+=1
        #106
        if item[1] =="JJR":
            print("JJR")
            JJR_cnt+=1
        #107
        if item[1] =="JJS":
            print("JJS")
            JJS_cnt+=1
        #108
        if item[1] =="MD":
            print("MD")
            MD_cnt+=1
        #109
        if item[1] =="NN":
            print("NN")
            NN_cnt+=1
        #110
        if item[1] =="NNS":
            print("NNS")
            NNS_cnt+=1
        #111
        if item[1] =="NNP":
            print("NNP")
            NNP_cnt+=1
        #112
        if item[1] =="NNPS":
            print("NNPS")
            NNPS_cnt+=1
        #113
        if item[1] =="POS":
            print("POS")
            POS_cnt+=1
        #114
        if item[1] =="PDT":
            print("PDT")
            PDT_cnt+=1
        #115
        if item[1] =="PRP":
            print("PRP")
            PRP_cnt+=1
        #116
        if item[1] =="PRP$":
            print("PRP$")
            PRP_dol_cnt+=1
        #117
        if item[1] =="RB":
            print("RB")
            RB_cnt+=1
        #118
        if item[1] =="RBR":
            print("RBR")
            RBR_cnt+=1
        #119
        if item[1] =="RBS":
            print("RBS")
            RBS_cnt+=1
        #120
        if item[1] =="TO":
            print("TO")
            TO_cnt+=1
        #121
        if item[1] =="VB":
            print("VB")
            VB_cnt+=1
        #122
        if item[1] =="VBD":
            print("VBD")
            VBD_cnt+=1
        #123
        if item[1] =="VBG":
            print("VBG")
            VBG_cnt+=1
        
        #124
        if item[1] =="VBN":
            print("VBN")
            VBN_cnt+=1
        #125
        if item[1] =="VBP":
            print("VBP")
            VBP_cnt+=1
        #126
        if item[1] =="VBZ":
            print("VBZ")
            VBZ_cnt+=1
        #127
        if item[1] =="WDT":
            print("WDT")
            WDT_cnt+=1
        #128
        if item[1] =="WP":
            print("WP")
            WP_cnt+=1
        #129
        if item[1] =="WP$":
            print("WP_dol")
            WP_dol_cnt+=1
        #130
        if item[1] =="WRB":
            print("WRB")
            WRB_cnt+=1
        
            
            
        

    repeat_word_cnt_lst.append(repeat_word_cnt)#1
    no_cnt_lst.append(no_cnt)#2
    never_cnt_lst.append(never_cnt)#3
    quite_cnt_lst.append(quite_cnt)#4
    but_cnt_lst.append(but_cnt)#5
    very_cnt_lst.append(very_cnt)#6
    more_cnt_lst.append(more_cnt)#7
    br_cnt_lst.append(br_cnt)#8
    else_cnt_lst.append(else_cnt)#9
    must_cnt_lst.append(must_cnt)#10
    many_cnt_lst.append(many_cnt)#11
    up_cnt_lst.append(up_cnt)#12
    so_cnt_lst.append(so_cnt)#13
    all_cnt_lst.append(all_cnt)#14
    dots_cnt_lst.append(dots_cnt)#15
    exclamation_mark_cnt_lst.append(exclamation_mark_cnt)#16
    question_mark_cnt_lst.append(question_mark_cnt)#17
    not_cnt_lst.append(not_cnt)#18
    er_cnt_lst.append(er_cnt)#19
    est_cnt_lst.append(est_cnt)#20
    ous_cnt_lst.append(ous_cnt)#21
    ly_cnt_lst.append(ly_cnt)#22
    dy_cnt_lst.append(dy_cnt)#23
    ful_cnt_lst.append(ful_cnt)#24
    ed_cnt_lst.append(ed_cnt)#25
    less_cnt_lst.append(less_cnt)#26
    re_cnt_lst.append(re_cnt) #27
    plus_cnt_lst.append(plus_cnt)#28
    minus_cnt_lst.append(minus_cnt)#29
    lol_cnt_lst.append(lol_cnt)#30


    CC_cnt_lst.append(CC_cnt)#101
    CD_cnt_lst.append(CD_cnt)#102
    DT_cnt_lst.append(DT_cnt)#103
    IN_cnt_lst.append(IN_cnt)#104
    JJ_cnt_lst.append(JJ_cnt)#105
    JJR_cnt_lst.append(JJR_cnt)#106
    JJS_cnt_lst.append(JJS_cnt)#107
    MD_cnt_lst.append(MD_cnt)#108
    NN_cnt_lst.append(NN_cnt)#109
    NNS_cnt_lst.append(NNS_cnt)#110
    NNP_cnt_lst.append(NNP_cnt)#111
    NNPS_cnt_lst.append(NNPS_cnt)#112
    POS_cnt_lst.append(POS_cnt)#113
    PDT_cnt_lst.append(PDT_cnt)#114
    PRP_cnt_lst.append(PRP_cnt)#115
    PRP_dol_cnt_lst.append(PRP_dol_cnt)#116
    RB_cnt_lst.append(RB_cnt)#117
    RBR_cnt_lst.append(RBR_cnt)#118
    RBS_cnt_lst.append(RBS_cnt)#119
    TO_cnt_lst.append(TO_cnt)#120
    VB_cnt_lst.append(VB_cnt)#121
    VBD_cnt_lst.append(VBD_cnt)#122
    VBG_cnt_lst.append(VBG_cnt)#123
    VBN_cnt_lst.append(NNPS_cnt)#124
    VBP_cnt_lst.append(POS_cnt)#125
    VBZ_cnt_lst.append(PDT_cnt)#126
    WDT_cnt_lst.append(PRP_cnt)#127
    WP_cnt_lst.append(PRP_dol_cnt)#128
    WP_dol_cnt_lst.append(RB_cnt)#129
    WRB_cnt_lst.append(RBR_cnt)#130

print(len(WRB_cnt_lst))

[('these', 'DT'), ('soft', 'JJ'), ('oatmeal', 'NN'), ('raisin', 'NN'), ('cookies', 'NNS'), ('are', 'VBP'), ('actually', 'RB'), ('quite', 'RB'), ('good', 'JJ'), ('.', '.')]
DT
DT
JJ
JJ
NN
NN
NN
NN
NNS
NNS
VBP
VBP
ly
RB
RB
quite
RB
RB
JJ
JJ
.
[('these', 'DT'), ('soft', 'JJ'), ('oatmeal', 'NN'), ('raisin', 'NN'), ('cookies', 'NNS')]
DT
DT
JJ
JJ
NN
NN
NN
NN
NNS
NNS
[('i', 'PRP'), ('even', 'RB'), ('shared', 'VBD'), ('it', 'PRP')]
PRP
PRP
RB
RB
ed
VBD
VBD
PRP
PRP
[('i', 'PRP'), ('even', 'RB'), ('shared', 'VBD'), ('it', 'PRP')]
PRP
PRP
RB
RB
ed
VBD
VBD
PRP
PRP
[('11', 'CD'), ('month', 'NN'), ('old', 'JJ'), (',', ',')]
CD
CD
NN
NN
JJ
JJ
,
[('i', 'PRP'), ("'m", 'VBP'), ('a', 'DT'), ('pretty', 'RB'), ('big', 'JJ'), ('health', 'NN'), ('nut', 'NN')]
PRP
PRP
VBP
VBP
DT
DT
RB
RB
JJ
JJ
NN
NN
NN
NN
[('a', 'DT'), ('pretty', 'RB'), ('big', 'JJ'), ('health', 'NN'), ('nut', 'NN')]
DT
DT
RB
RB
JJ
JJ
NN
NN
NN
NN
[('i', 'PRP'), ('feed', 'VBP'), ('my', 'PRP$'), ('daughter', 'NN'), ('.', '.')]
PRP
PRP
VBP
VBP


VB
PRP
PRP
IN
IN
DT
DT
NN
NN
.
[('these', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('rich', 'JJ')]
DT
DT
NNS
NNS
VBP
VBP
JJ
JJ
[('flavor', 'NN'), (',', ','), ('not', 'RB'), ('dry', 'JJ'), (',', ','), ('and', 'CC')]
NN
NN
,
not
RB
RB
JJ
JJ
,
CC
CC
[('a', 'DT'), ('great', 'JJ'), ('cookie', 'NN'), ('.', '.')]
DT
DT
JJ
JJ
NN
NN
.
[('it', 'PRP'), ('was', 'VBD'), ('delicious', 'JJ'), ('.', '.')]
PRP
PRP
VBD
VBD
JJ
JJ
.
[('want', 'VBP'), ('snacks', 'NNS'), ('yet', 'RB')]
VBP
VBP
NNS
NNS
RB
RB
[('the', 'DT'), ('seagulls', 'NNS'), ('attacking', 'VBG'), ('open', 'JJ'), ('food', 'NN'), ('bags', 'NNS'), ('.', '.')]
DT
DT
NNS
NNS
VBG
VBG
JJ
JJ
NN
NN
NNS
NNS
.
[('attacking', 'VBG'), ('open', 'JJ'), ('food', 'NN'), ('bags', 'NNS'), ('.', '.')]
VBG
VBG
JJ
JJ
NN
NN
NNS
NNS
.
[('i', 'PRP'), ('am', 'VBP'), ('pretty', 'RB'), ('picky', 'JJ')]
PRP
PRP
VBP
VBP
RB
RB
JJ
JJ
[('like', 'VBP'), ('homemade', 'JJ'), ('cookies', 'NNS'), ('.', '.')]
VBP
VBP
JJ
JJ
NNS
NNS
.
[('it', 'PRP'), ('was', 'VBD'), ('a', 'DT'

NN
NN
[('sent', 'VBN'), ('to', 'TO'), ('test', 'VB')]
VBN
VBN
TO
TO
VB
VB
[('it', 'PRP'), ('lasted', 'VBD'), ('less', 'RBR'), ('then', 'RB'), ('ten', 'VB'), ('minutes', 'NNS'), ('in', 'IN')]
PRP
PRP
ed
VBD
VBD
less
RBR
RBR
RB
RB
VB
VB
NNS
NNS
IN
IN
[('lasted', 'VBD'), ('less', 'RBR'), ('then', 'RB'), ('ten', 'VB'), ('minutes', 'NNS')]
ed
VBD
VBD
less
RBR
RBR
RB
RB
VB
VB
NNS
NNS
[('any', 'DT'), ('sweet', 'JJ'), ('tooth', 'NN'), ('craving', 'VBG')]
DT
DT
JJ
JJ
NN
NN
VBG
VBG
[('any', 'DT'), ('sweet', 'JJ'), ('tooth', 'NN')]
DT
DT
JJ
JJ
NN
NN
[('other', 'JJ'), ('desserts', 'NNS'), ('.', '.')]
er
JJ
JJ
NNS
NNS
.
[('he', 'PRP'), ('had', 'VBD'), ('no', 'DT'), ('clue', 'NN'), ('there', 'EX'), ('were', 'VBD'), ('raisins', 'NNS')]
PRP
PRP
VBD
VBD
no
DT
DT
NN
NN
EX
VBD
VBD
NNS
NNS
[('there', 'EX'), ('were', 'VBD'), ('raisins', 'NNS')]
EX
VBD
VBD
NNS
NNS
[('they', 'PRP'), ('were', 'VBD'), ('just', 'RB'), ('that', 'IN'), ('yummy', 'NN'), ('(', '('), (':', ':')]
PRP
PRP
VBD
VBD
RB
RB
IN
IN
NN
NN
(
:

JJ
JJ
RB
RB
IN
IN
[('split', 'VB'), ('it', 'PRP'), ('between', 'IN'), ('both', 'DT'), ('kids', 'NNS'), ('.', '.')]
VB
VB
PRP
PRP
IN
IN
DT
DT
NNS
NNS
.
[('a', 'DT'), ('great', 'JJ'), ('special', 'JJ'), ('treat', 'NN')]
DT
DT
JJ
JJ
JJ
JJ
NN
NN
[('get', 'VB'), ('that', 'DT'), ('extra', 'JJ'), ('fiber', 'NN'), ('we', 'PRP'), ('all', 'DT')]
VB
VB
DT
DT
JJ
JJ
NN
NN
PRP
PRP
all
DT
DT
[('get', 'VB'), ('that', 'DT'), ('extra', 'JJ'), ('fiber', 'NN'), ('we', 'PRP')]
VB
VB
DT
DT
JJ
JJ
NN
NN
PRP
PRP
[('that', 'DT'), ('extra', 'JJ'), ('fiber', 'NN')]
DT
DT
JJ
JJ
NN
NN
[('i', 'PRP'), ('buy', 'VBP'), ('them', 'PRP')]
PRP
PRP
VBP
VBP
PRP
PRP
[('i', 'PRP'), ('buy', 'VBP'), ('them', 'PRP')]
PRP
PRP
VBP
VBP
PRP
PRP
[('quick', 'JJ'), ('snacks', 'NNS'), (',', ',')]
JJ
JJ
NNS
NNS
,
[('taste', 'NN'), ('sweet', 'NN'), ('and', 'CC'), ('delicious', 'JJ'), ('but', 'CC'), ('are', 'VBP'), ('healthier', 'JJR'), ('than', 'IN')]
NN
NN
NN
NN
CC
CC
JJ
JJ
but
CC
CC
VBP
VBP
er
JJR
JJR
IN
IN
[('taste', 'NN'), ('sweet', 'N

VBD
PRP
PRP
RP
[('we', 'PRP'), ('enjoyed', 'VBD'), ('them', 'PRP')]
PRP
PRP
ed
VBD
VBD
PRP
PRP
[('i', 'PRP'), ('received', 'VBD'), ('this', 'DT'), ('cookie', 'NN')]
PRP
PRP
ed
re
VBD
VBD
DT
DT
NN
NN
[('my', 'PRP$'), ('moms', 'NNS'), ('voxbox', 'VBP'), ('from', 'IN'), ('influenster.com', 'NN'), ('.', '.')]
PRP$
PRP$
NNS
NNS
VBP
VBP
IN
IN
NN
NN
.
[('these', 'DT'), ('make', 'VBP'), ('a', 'DT'), ('great', 'JJ'), ('snack', 'NN')]
DT
DT
VBP
VBP
DT
DT
JJ
JJ
NN
NN
[('a', 'DT'), ('great', 'JJ'), ('snack', 'NN')]
DT
DT
JJ
JJ
NN
NN
[('kids', 'NNS'), ('and', 'CC'), ('adults', 'NNS'), ('alike', 'RB'), ('.', '.')]
NNS
NNS
CC
CC
NNS
NNS
RB
RB
.
[('really', 'RB'), ('dislike', 'VB'), ('oatmeal', 'NN'), ('cookies', 'NNS')]
ly
re
RB
RB
VB
VB
NN
NN
NNS
NNS
[('raisins..i', 'NN'), ('got', 'VBD'), ('a', 'DT'), ('cookie', 'NN'), ('from', 'IN'), ('influenster', 'NN'), ('and', 'CC')]
NN
NN
VBD
VBD
DT
DT
NN
NN
IN
IN
NN
NN
CC
CC
[('raisins..i', 'NN'), ('got', 'VBD'), ('a', 'DT'), ('cookie', 'NN')]
NN
NN
VBD
VBD
D

...

DT
er
JJR
JJR
NN
NN
[('a', 'DT'), ('healthier', 'JJR'), ('choice', 'NN')]
DT
DT
er
JJR
JJR
NN
NN
[('is', 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('feeling', 'NN'), ('.', '.')]
VBZ
VBZ
DT
DT
JJ
JJ
NN
NN
.
[('is', 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('feeling', 'NN'), ('.', '.')]
VBZ
VBZ
DT
DT
JJ
JJ
NN
NN
.
[('a', 'DT'), ('good', 'JJ'), ('feeling', 'NN'), ('.', '.')]
DT
DT
JJ
JJ
NN
NN
.
[('you', 'PRP'), ('are', 'VBP'), ('looking', 'VBG'), ('for', 'IN'), ('a', 'DT'), ('cookie', 'NN')]
PRP
PRP
VBP
VBP
VBG
VBG
IN
IN
DT
DT
NN
NN
[('makes', 'VBZ'), ('look', 'VB'), ('no', 'DT'), ('further', 'NN'), ('.', '.')]
VBZ
VBZ
VB
VB
no
DT
DT
NN
NN
.
[('makes', 'VBZ'), ('look', 'VB'), ('no', 'DT'), ('further', 'NN'), ('.', '.')]
VBZ
VBZ
VB
VB
no
DT
DT
NN
NN
.
[('toddler', 'NN'), ('loves', 'VBZ'), ('them', 'PRP'), ('too', 'RB'), ('!', '.')]
NN
NN
VBZ
VBZ
PRP
PRP
RB
RB
!
.
[('loves', 'VBZ'), ('them', 'PRP'), ('too', 'RB'), ('!', '.')]
VBZ
VBZ
PRP
PRP
RB
RB
!
.
[('it', 'PRP'), ('is', 'VBZ'), ('good', 'JJ'), (

NN
NN
.
[('had', 'VBD'), ('a', 'DT'), ('detectable', 'JJ'), ('raisin', 'NN'), ('flavor', 'NN'), ('.', '.')]
VBD
VBD
DT
DT
JJ
JJ
NN
NN
NN
NN
.
[('a', 'DT'), ('detectable', 'JJ'), ('raisin', 'NN'), ('flavor', 'NN'), ('.', '.')]
DT
DT
JJ
JJ
NN
NN
NN
NN
.
[('they', 'PRP'), ('come', 'VBP'), ('individually', 'RB'), ('wrapped', 'VBN'), (',', ',')]
PRP
PRP
VBP
VBP
ly
RB
RB
VBN
VBN
,
[('little', 'JJ'), ('ones', 'NNS'), ('.', '.')]
JJ
JJ
NNS
NNS
.
[('bubs', 'NNP'), ('is', 'VBZ'), ('allergic', 'JJ')]
NNP
NNP
VBZ
VBZ
JJ
JJ
[('these', 'DT'), ('are', 'VBP'), ('delicious', 'JJ'), (',', ',')]
DT
DT
VBP
VBP
JJ
JJ
,
[('you', 'PRP'), ('wo', 'MD'), ('not', 'RB'), ('be', 'VB')]
PRP
PRP
MD
MD
not
RB
RB
VB
VB
[('i', 'PRP'), ('did', 'VBD'), ('try', 'VB'), ('the', 'DT'), ('cookie', 'NN')]
PRP
PRP
VBD
VBD
VB
VB
DT
DT
NN
NN
[('and', 'CC'), ('it', 'PRP'), ('was', 'VBD'), ('so', 'RB'), ('soft', 'JJ'), ('and', 'CC'), ('melted', 'VBN'), ('in', 'IN'), ('my', 'PRP$')]
CC
CC
PRP
PRP
VBD
VBD
so
RB
RB
JJ
JJ
CC
CC
VBN
VBN

PRP
PRP
VBP
VBP
ed
JJ
JJ
PRP$
PRP$
NN
NN
ed
VBD
VBD
[('these', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('very', 'RB'), ('delicious', 'JJ'), ('!', '.')]
DT
DT
NNS
NNS
VBP
VBP
very
RB
RB
JJ
JJ
!
.
[('getting', 'VBG'), ('this', 'DT'), ('cookie', 'NN'), ('i', 'PRP'), ('ate', 'VBP'), ('it', 'PRP')]
VBG
VBG
DT
DT
NN
NN
PRP
PRP
VBP
VBP
PRP
PRP
[('getting', 'VBG'), ('this', 'DT'), ('cookie', 'NN'), ('i', 'PRP')]
VBG
VBG
DT
DT
NN
NN
PRP
PRP
[('a', 'DT'), ('little', 'JJ'), ('dry', 'JJ'), (',', ','), ('but', 'CC')]
DT
DT
JJ
JJ
JJ
JJ
,
but
CC
CC
[('a', 'DT'), ('little', 'JJ'), ('dry', 'JJ'), (',', ',')]
DT
DT
JJ
JJ
JJ
JJ
,
[('the', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('a', 'DT'), ('good', 'JJ'), ('snack', 'NN')]
DT
DT
NNS
NNS
VBP
VBP
DT
DT
JJ
JJ
NN
NN
[('a', 'DT'), ('good', 'JJ'), ('snack', 'NN')]
DT
DT
JJ
JJ
NN
NN
[('are', 'VBP'), ('not', 'RB'), ('super', 'JJ'), ('high', 'NN')]
VBP
VBP
not
RB
RB
er
JJ
JJ
NN
NN
[('i', 'PRP'), ('received', 'VBD'), ('this', 'DT'), ('product', 'NN'), ('comp

VBP
DT
DT
NNP
NNP
NNP
NNP
NNP
NNP
NNS
NNS
.
[('i', 'VB'), ('enjoyed', 'VBP'), ('the', 'DT'), ('quaker', 'NNP'), ('soft', 'NNP'), ('baked', 'NNP'), ('cookies', 'NNS'), ('.', '.')]
VB
VB
VBP
VBP
DT
DT
NNP
NNP
NNP
NNP
NNP
NNP
NNS
NNS
.
[('i', 'VB'), ('enjoyed', 'VBP'), ('the', 'DT'), ('quaker', 'NNP'), ('soft', 'NNP'), ('baked', 'NNP'), ('cookies', 'NNS'), ('.', '.')]
VB
VB
VBP
VBP
DT
DT
NNP
NNP
NNP
NNP
NNP
NNP
NNS
NNS
.
[('buy', 'VB'), ('more', 'JJR'), ('!', '.')]
VB
VB
more
JJR
JJR
!
.
[('i', 'PRP'), ('got', 'VBD'), ('the', 'DT'), ('influenster', 'NN'), ('box', 'NN')]
PRP
PRP
VBD
VBD
DT
DT
NN
NN
NN
NN
[('it', 'PRP'), ('had', 'VBD'), ('several', 'JJ'), ('products', 'NNS'), ('in', 'IN')]
PRP
PRP
VBD
VBD
JJ
JJ
NNS
NNS
IN
IN
[('had', 'VBD'), ('several', 'JJ'), ('products', 'NNS')]
VBD
VBD
JJ
JJ
NNS
NNS
[('loved', 'VBD'), ('them', 'PRP'), ('.', '.')]
ed
VBD
VBD
PRP
PRP
.
[('loved', 'VBD'), ('them', 'PRP'), ('.', '.')]
ed
VBD
VBD
PRP
PRP
.
[('i', 'PRP'), ('received', 'VBD'), ('the', 'DT'), ('

TO
TO
VB
VB
more
JJR
JJR
[('their', 'PRP$'), ('different', 'JJ'), ('flavors', 'NNS'), (',', ','), ('so', 'RB'), ('yum', 'JJ'), ('!', '.')]
PRP$
PRP$
JJ
JJ
NNS
NNS
,
so
RB
RB
JJ
JJ
!
.
[('their', 'PRP$'), ('different', 'JJ'), ('flavors', 'NNS'), (',', ',')]
PRP$
PRP$
JJ
JJ
NNS
NNS
,
[('i', 'PRP'), ('mostly', 'RB'), ('make', 'VBP'), ('my', 'PRP$'), ('own', 'JJ'), ('cookies', 'NNS'), (',', ',')]
PRP
PRP
ly
RB
RB
VBP
VBP
PRP$
PRP$
JJ
JJ
NNS
NNS
,
[('i', 'PRP'), ('mostly', 'RB'), ('make', 'VBP'), ('my', 'PRP$'), ('own', 'JJ'), ('cookies', 'NNS'), (',', ',')]
PRP
PRP
ly
RB
RB
VBP
VBP
PRP$
PRP$
JJ
JJ
NNS
NNS
,
[('my', 'PRP$'), ('own', 'JJ'), ('cookies', 'NNS'), (',', ',')]
PRP$
PRP$
JJ
JJ
NNS
NNS
,
[('i', 'PRP'), ('would', 'MD'), ('definitely', 'RB'), ('purchase', 'VB'), ('these', 'DT')]
PRP
PRP
MD
MD
ly
RB
RB
VB
VB
DT
DT
[('i', 'PRP'), ('received', 'VBD'), ('a', 'DT'), ('free', 'JJ'), ('sample', 'NN'), ('from', 'IN'), ('influenster', 'NNP'), (',', ','), ('and', 'CC')]
PRP
PRP
ed
re
VBD
VBD
D

### Combining all the features into one Table and write to CSV

In [15]:
df_train = pd.DataFrame(list(zip(df_tup_lst,
                      df_syn0_lst,
                      df_syn_all_lst,
                     repeat_word_cnt_lst,
                        no_cnt_lst,
                        never_cnt_lst,
                        quite_cnt_lst,
                        but_cnt_lst,
                        very_cnt_lst,
                        more_cnt_lst,
                        br_cnt_lst,
                        else_cnt_lst,
                        must_cnt_lst,
                        many_cnt_lst,
                        up_cnt_lst,
                        so_cnt_lst,
                        all_cnt_lst,
                        dots_cnt_lst,
                        exclamation_mark_cnt_lst,
                        question_mark_cnt_lst,
                        not_cnt_lst,
                        er_cnt_lst,
                        est_cnt_lst,
                        ous_cnt_lst,
                        ly_cnt_lst,
                        dy_cnt_lst,
                        ful_cnt_lst,
                        ed_cnt_lst,
                        less_cnt_lst,
                        CC_cnt_lst,
                        CD_cnt_lst,
                        DT_cnt_lst,
                        IN_cnt_lst,
                        JJ_cnt_lst,
                        JJR_cnt_lst,
                        JJS_cnt_lst,
                        MD_cnt_lst,
                        NN_cnt_lst,
                        NNS_cnt_lst,
                        NNP_cnt_lst,
                        NNPS_cnt_lst,
                        POS_cnt_lst,
                        PDT_cnt_lst,
                        PRP_cnt_lst,
                        PRP_dol_cnt_lst,
                        RB_cnt_lst,
                        RBR_cnt_lst,
                        RBS_cnt_lst,
                        TO_cnt_lst,
                        VB_cnt_lst,
                        VBD_cnt_lst,
                        VBG_cnt_lst,
                        VBN_cnt_lst,
                        VBP_cnt_lst,
                        VBZ_cnt_lst,
                        WDT_cnt_lst,
                        WP_cnt_lst,
                        WP_dol_cnt_lst,
                        WRB_cnt_lst))
             , columns=["segments","syn0", "syn_all","repeat_word_cnt"
                                                            ,"no_cnt"
                                                            ,"never_cnt"
                                                            ,"quite_cnt"
                                                            ,"but_cnt"
                                                            ,"very_cnt"
                                                            ,"more_cnt"
                                                            ,"br_cnt"
                                                            ,"else_cnt"
                                                            ,"must_cnt"
                                                            ,"many_cnt"
                                                            ,"up_cnt"
                                                            ,"so_cnt"
                                                            ,"all_cnt"
                                                            ,"dots_cnt"
                                                            ,"exclamation_mark_cnt"
                                                            ,"question_mark_cnt"
                                                            ,"not_cnt"
                                                            ,"er_cnt"
                                                            ,"est_cnt"
                                                            ,"ous_cnt"
                                                            ,"ly_cnt"
                                                            ,"dy_cnt"
                                                            ,"ful_cnt"
                                                            ,"ed_cnt"
                                                            ,"less_cnt"


                                                            ,"CC_cnt"
                                                            ,"CD_cnt"
                                                            ,"DT_cnt"
                                                            ,"IN_cnt"
                                                            ,"JJ_cnt"
                                                            ,"JJR_cnt"
                                                            ,"JJS_cnt"
                                                            ,"MD_cnt"
                                                            ,"NN_cnt"
                                                            ,"NNS_cnt"
                                                            ,"NNP_cnt"
                                                            ,"NNPS_cnt"
                                                            ,"POS_cnt"
                                                            ,"PDT_cnt"
                                                            ,"PRP_cnt"
                                                            ,"PRP_dol_cnt"
                                                            ,"RB_cnt"
                                                            ,"RBR_cnt"
                                                            ,"RBS_cnt"
                                                            ,"TO_cnt"
                                                            ,"VB_cnt"
                                                            ,"VBD_cnt"
                                                            ,"VBG_cnt"
                                                            ,"VBN_cnt"
                                                            ,"VBP_cnt"
                                                            ,"VBZ_cnt"
                                                            ,"WDT_cnt"
                                                            ,"WP_cnt"
                                                            ,"WP_dol_cnt"
                                                            ,"WRB_cnt"])

#Create the Training Dataset
df_train2.to_csv("final_train_table.csv", index=False)

In [16]:
df_train.shape

(6593, 59)

### After manually labeling the class

In [20]:
df_train= pd.read_csv("train1.csv", index_col=False)
print(df_train.shape)
df_train.head(3)

(1064, 60)


,class,segments,syn0,syn_all,repeat_word_cnt,no_cnt,never_cnt,quite_cnt,but_cnt,very_cnt,...,VB_cnt,VBD_cnt,VBG_cnt,VBN_cnt,VBP_cnt,VBZ_cnt,WDT_cnt,WP_cnt,WP_dol_cnt,WRB_cnt
0,1,"[('these', 'DT'), ('soft', 'JJ'), ('oatmeal', ...",0.017857,0.045292,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
1,1,"[('these', 'DT'), ('soft', 'JJ'), ('oatmeal', ...",-0.093750,-0.105263,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"[('i', 'PRP'), ('even', 'RB'), ('shared', 'VBD...",0.125000,0.012500,0,0,0,0,0,0,...,0,1,0,0,0,0,2,0,1,0


### Classifier (Try 1)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

In [75]:
Y = np.array(df_train["class"])
X = np.array(df_train.iloc[:,2:])
print(X.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)

print(Counter(y_train))
print(Counter(y_test))


(1064, 58)
(1064,)
Counter({1: 549, 0: 163})
Counter({1: 253, 0: 99})


In [76]:
#Model
model = BernoulliNB()
model.fit(x_train,y_train)

#Prediction
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
pred_proba_test = model.predict_proba(x_test)

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
#prob_cutoff=0.5
#pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
#cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
#print(cm_test)

[[ 68  95]
 [ 45 504]]
Recall for Training Set:  0.9180327868852459
Precision for Training Set:  0.8414023372287145
Accuracy for Training Set:  0.8033707865168539

[[ 43  56]
 [ 23 230]]
Recall for Testing Set:  0.9090909090909091
Precision for Testing Set:  0.8041958041958042
Accuracy for Training Set:  0.7755681818181818


### Attempt to correct more "False Positive" classification to boost the Accuracy by Intensitify Polarity Score for Adverbs

#### Convert the String Segment in training set back to tuples list

In [332]:

new_lst2=[]
for i in range(0,df_train.shape[0]):

    new_lst = []
    line = df_train.segments.iloc[i]
    #print(line)
    #print()
    line = line[3:-3]
    line = line.replace('"',"'")
    #print(line)
    line = line.replace("'), ('","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.replace("', '","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.split("ashgh582E@#$$")
    #print(line)
    #print()
    
    j=0
    while j < len(line):
        new_lst.append(tuple((line[j],line[j+1])))
        j+=2
    #print(new_lst)
    #print()
    new_lst2.append(new_lst)

#### Calculate the Polarity Score with Adverb Booster

In [416]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
#df_lst = [0 for item in range(len(review_lst2))]
df_syn0_lst2 = []
df_syn_all_lst2 = []
df_tup_lst2 = []
ADV_booster=1.8 #Adv Booster Value
cnt=0



#for tups in review_lst[0:10]:
for tups in new_lst2:
    if len(tups)<3:
        continue
    #print(tups)
    lst=[]
    tot_pscore_syn0=0
    tot_pscore_syn_all = 0
    word_cnt=0
    
    for i in range(len(tups)):
        #print("tups[i]",tups[i])
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        #print("w",w)
        #print("tag",tag)
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print("1")
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                #print("2")
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                #print("3")
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            #print(synsets)
            #print(synset)
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            #print(swn_synset)
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            #print(pscore)
            
            #Adverb booster
            if wntag == wn.ADJ:
                pscore = pscore * ADV_booster
                #print(wntag)
            
            tot_pscore_syn0 += pscore
            
            tot_pscore_syn=0
            if len(synsets)>1:
                for synset in synsets:
                    swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                    #print(swn_synset)
                    pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                    #print("pscore",pscore)
                    
                    #Adverb booster
                    if wntag == wn.ADJ:
                        pscore = pscore * ADV_booster
                        #print(wntag)
                    
                    tot_pscore_syn += pscore
                tot_pscore_syn_all += (tot_pscore_syn/len(synsets))
                #print("tot_pscore_syn_all",tot_pscore_syn_all)
            else: tot_pscore_syn_all += tot_pscore_syn0
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
    
    if word_cnt == 0:
        df_syn0_lst2.append(0)
        df_syn_all_lst2.append(0)
        #print("Word count",word_cnt)
        df_tup_lst2.append(tups)
        cnt+=1
    else:
        df_syn0_lst2.append(tot_pscore_syn0/word_cnt)
        df_syn_all_lst2.append(tot_pscore_syn_all/word_cnt)
        #print("Word count",word_cnt)
        df_tup_lst2.append(tups)
        cnt+=1
        
print(len(df_syn0_lst2), len(df_syn_all_lst2), len(df_tup_lst2))
    



1064 1064 1064


### Classifier 2 (Adverb booster)

In [417]:
X2=X.copy()

#X2[:,2:].shape
lst1= [[item] for item in df_syn0_lst2]
lst2= [[item] for item in df_syn_all_lst2]

X2 = np.concatenate((X2[:,2:],lst1,lst2), axis=1)
print("Before edit Polarity by Booster",X.shape)
print("After edit Polarity by Booster",X2.shape)

Before edit Polarity by Booster (1064, 58)
After edit Polarity by Booster (1064, 58)


In [424]:
print(X2.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X2, Y, test_size=0.33, random_state=1)

print(Counter(y_train))
print(Counter(y_test))



#Model
model = BernoulliNB()
model.fit(x_train,y_train)

#Prediction
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
pred_proba_test = model.predict_proba(x_test)

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
prob_cutoff=0.4
pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
print(cm_test)

(1064, 58)
(1064,)
Counter({1: 549, 0: 163})
Counter({1: 253, 0: 99})
[[ 68  95]
 [ 45 504]]
Recall for Training Set:  0.9180327868852459
Precision for Training Set:  0.8414023372287145
Accuracy for Training Set:  0.8033707865168539

[[ 43  56]
 [ 23 230]]
Recall for Testing Set:  0.9090909090909091
Precision for Testing Set:  0.8041958041958042
Accuracy for Training Set:  0.7755681818181818
[[ 33  66]
 [ 15 238]]


In [384]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [397]:
bb = [[item] for item in [4,4,4]]
bb

[[4], [4], [4]]

In [91]:

dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
dog.path_similarity(cat)

0.2

In [161]:
syn_good = wn.synsets("good", pos=wntag)[0]
syn_declicious = wn.synsets("delicious")[0]
syn_excellent = wn.synsets("excellent")[0]
syn_great = wn.synsets("great", pos=wntag)[0]

syn_w = wn.synsets("bad", pos=wntag)[0]
#syn_w = wn.synsets("delicious", pos=wntag)[0]

syn_good.path_similarity(syn_great)
#syn_excellent
#syn_great
#syn_good

syn_good_n = wn.synsets("good", pos=wntag)[0]
syn_good_a = wn.synsets("good", pos=wntag)[4]
syn_good_s = wn.synsets("good", pos=wntag)[10]

wn.synsets("delicious")

#wn.synsets("excellent", pos=wntag)
wn.synsets("excellent")

[Synset('excellent.s.01')]

In [63]:

import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
label_lst = []
avg_label_score_lst = []


cp = nltk.RegexpParser(grammar)


start=time.time()
token_line_lst = []
cnt=0

sents=["I love this food. I really like it"]

for line in sents:
    token_line_lst=[]
    print(line)
    for token_line in sent_tokenize(line):
        print(token_line)
        token_line_lst.append(token_line)
    
    sentlist = [word_tokenize(line) for line in token_line_lst]
    print(sentlist)
    
    review_lst=[]    
    for sent in sentlist: #Iterate the Review sentlist 
        tree = cp.parse(nltk.pos_tag(sent)) #pos_tag the sentence and parse the sentence
        #print(tree)
        for subtree in tree.subtrees():
            if subtree.label() =="NP":
                #print("NP")
                #print(subtree.leaves())    
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause1a":
                #print("clause1a")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause1b":
                #print("clause1b")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause2a":
                #print("clause2a")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause2b":
                #print("clause2b")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause3":
                #print("clause3")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause4":
                #print("clause4")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause5":
                #print("clause5")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
        
    #Filtering out all the segments which do not have feature words              
    review_lst2=[] #initialize a new list
    for line in review_lst: #iterating the list with extracted segments
        for item in line:
            if item[0] in feature_lst: #Feature list is obtained in Step 3
                review_lst2.append(line) #Append all the segments have feature words in the new list
                continue
        
    

##################
    #print(len(review_lst))
    #print(review_lst)
    #print("# of segments",len(review_lst2))
    #print(review_lst2)
##################
    df_lst1=[]
    df_lst2=[]

    for tups in review_lst2:
        lst=[]
        tot_pscore=0
        word_cnt=0
        
        for i in range(len(tups)):
            tups[i]=list(tups[i]) #Convert the list back to tuples
            w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
            tag=tups[i][1] #Assign the tag from the 2nd element of tuples

            #If the word is a negative contraction suffix ('negcontractions'), 
            #change it to the word: "not"
            if w in negcontractions:
                w="not"
                
            # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
            wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
            if wntag is not None and (w not in stopwords or w in negwords): 
                #print(w)
                tups[i][0] = w 
                word_cnt+=1
                lemma = wn_lemmatize(w, wntag) #lemmatize token

                if lemma is None:
                    tups[i]=tuple(tups[i])
                    continue

                synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

                if not synsets:
                    tups[i]=tuple(tups[i])
                    continue

                # take the first sense
                synset = synsets[0]
                swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                tot_pscore+=pscore

                #Convert the list back to tuples
                tups[i]=tuple(tups[i])

            else:
                tups[i][0] = w 
                tups[i]=tuple(tups[i])

        if word_cnt == 0: df_lst1.append(0)
        else: df_lst1.append(tot_pscore/word_cnt)
        #print("df_lst1:",df_lst1)


        df_lst2.append(tups)

    
    if np.mean(df_lst1) > 0 :
        label=1
    else:
        label=0
    #print(df_lst2)
    avg_label_score_lst.append(np.mean(df_lst1))
    label_lst.append(label)
   
    print(cnt)
    cnt+=1
end=time.time()
print("%d, Time taken %f min"%(cnt+1, ((end-start)/ 60)))
#print("avg",avg_label_score_lst)    
#print("label",label_lst)
print(len(label_lst))

I love this food. I really like it
I love this food.
I really like it
[['I', 'love', 'this', 'food', '.'], ['I', 'really', 'like', 'it']]
0
2, Time taken 0.000164 min
1


/Users/KevQuant/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/KevQuant/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [119]:
avg_label_score_lst

NameError: name 'avg_label_score_lst' is not defined

In [131]:
y__true = df.Score.copy()
#y__pred = label_lst
y__true[y__true < 3] =0
y__true[y__true > 3] =1


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y__true,y__pred,labels=[0,1,3])
cm
#(cm[i,i] / sum(cm[i]))

array([[ 52429,  29608,      0],
       [223949, 219828,      0],
       [ 22287,  20353,      0]])

In [130]:
Counter(y__true)




Counter({0: 82037, 1: 443777, 3: 42640})

In [116]:
len(label_lst)
df.Score.head(5)

0    5
1    1
2    4
3    2
4    5
Name: Score, dtype: int64

In [105]:
df_pred = df.copy()
df_pred.shape

(568454, 10)

In [112]:
pred_col = pd.DataFrame(label_lst, columns=["pred"])
#pd.concat((df_pred, pred_col))
pred_col

,pred
0,1
1,0
2,1
3,0
4,0
5,1
6,0
7,1
8,0
9,1


### Step-6: Polarity Score

##### Given Functions

In [ ]:
w="comfortable"
tag="JJ"
wntag = to_wntag(tag)
lemma = wn_lemmatize(w, wntag)
synsets = wn.synsets(lemma, pos=wntag)
print(synsets)
synset = synsets[2]


swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
print(swn_synset)
pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
pscore

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
df_lst = [0 for item in range(len(review_lst2))]
df_lst2 = []
df_lst1 = []

cnt=0
for tups in review_lst2:
    lst=[]
    tot_pscore=0
    word_cnt=0
    for i in range(len(tups)):
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print(w)
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            tot_pscore+=pscore
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
            
    df_lst1.append(tot_pscore/word_cnt)
    df_lst2.append(tups)
    cnt+=1
    
    
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
df2= pd.DataFrame(df_lst2)
df = pd.concat((df1,df2), axis=1)
df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 
                    ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

In [ ]:
len(review_lst2)

In [ ]:
pre_lst=[]
new_lst=[]
cnt=0
with open("XXXX-rooms.txt", "w") as f:
    for row in range(0,df.shape[0]):
        new_lst = list(df.iloc[row])
        new_lst = [item  for item in new_lst if item is not None]
        #print(new_lst)

        if pre_lst == new_lst:
            continue
            
        for item in df.iloc[row]:
            if item is not None:
                try:
                    if type(float(item)) == float:
                        #print(item, end=", [")
                        f.write("{0}, [".format(item))
                        continue
                except:
                    pass
                #print(item, end=",")
                f.write("{0}, ".format(item))

            else:
                #print("]",end="\n")
                f.write("]\n")
                cnt+=1
                break
        pre_lst = new_lst
f.close()
print(cnt)    

########################END

In [ ]:
#sent = nltk.pos_tag(nltk.word_tokenize("She girl let down her long hair"))
sent = nltk.pos_tag(nltk.word_tokenize("quick"))
print(sent)

#regex grammar
grammar = r"""
NP:{<DT|PRP\$>?<JJ>*<NN>} 
FOO:{<NP><VB.><JJ>}"""
#"NP:{DT|PRP\$>?<JJ>*<NN>}" #, FOO: {<NP><VB.><JJ>}"

#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)



In [ ]:
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)


In [ ]:
for subtree in tree.subtrees():
    if subtree.label() =="NP":
        print(subtree.leaves())

In [ ]:
for sent in sentlist:
    cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":
            print(subtree.leaves())